<a href="https://colab.research.google.com/github/merishnaSuwal/nep-off-langdetect/blob/main/NepNER_final.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Named Entity Recognition for Offensive Nepali Language

We are performing named entity recognition task in Nepali language. This notebook runs completely standalone.

Warning: This notebook contains offensive language.


## Installation

In [1]:
%%capture
!python3 -m pip install -U huggingface_hub
!python3 -m pip install -U transformers
!python3 -m pip install -U datasets evaluate
!python3 -m pip install -U accelerate
!python3 -m pip install -U seqeval

# Data Preprocessing

## Load NepSA dataset from github clone


In [2]:
# Your github repo link here
!git clone https://github.com/merishnaSuwal/nep-off-langdetect

Cloning into 'nep-off-langdetect'...
remote: Enumerating objects: 44, done.
remote: Counting objects: 100% (44/44), done.
remote: Compressing objects: 100% (30/30), done.
remote: Total 44 (delta 13), reused 35 (delta 10), pack-reused 0
Receiving objects: 100% (44/44), 603.64 KiB | 9.43 MiB/s, done.
Resolving deltas: 100% (13/13), done.


In [3]:
DATA_DIR = 'nep-off-langdetect/NepNER/dataset'

In [4]:
import os
from datasets import load_dataset

data_files = {
    "train": "train.txt",
    "validation": "valid.txt",
    "test": "test.txt",
}

load_script = os.path.join(DATA_DIR, "load_ner.py")

raw_datasets = load_dataset(load_script, data_files=data_files)

Extracting data files:   0%|          | 0/3 [00:00<?, ?it/s]

Generating train split: 0 examples [00:00, ? examples/s]

Generating validation split: 0 examples [00:00, ? examples/s]

Generating test split: 0 examples [00:00, ? examples/s]

Check the basic information on the loaded dataset

In [5]:
raw_datasets

DatasetDict({
    train: Dataset({
        features: ['id', 'tokens', 'ner_tags'],
        num_rows: 2323
    })
    validation: Dataset({
        features: ['id', 'tokens', 'ner_tags'],
        num_rows: 330
    })
    test: Dataset({
        features: ['id', 'tokens', 'ner_tags'],
        num_rows: 280
    })
})

Check sample of tokens from train dataset

In [ ]:
print(raw_datasets["train"][0]["tokens"])

['भालुनी', 'सावित्री', 'कुकुरनी', 'मिले', 'को', 'रहेछ', 'आजा', 'प्रक्षया', 'थाहा', 'भयो', 'निर्माल', 'बहिनी', 'लाई', 'बलत्कार', 'गर्न', 'लगाउने', 'यनि', 'भलु', 'हरु', 'रहेछ', 'पहिला', 'जाती', 'आन्दोलान', 'गरे', 'को', 'थियो', 'त्यो', 'सबै', 'यनि', 'हरु', 'को', 'नाटक', 'रहेछ', 'आजा', 'बल', 'थाहा', 'भयो', '।']


Check the NER tags (its IDS) of the corresponding sample

In [ ]:
print(raw_datasets["train"][0]["ner_tags"])

[1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 7, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]


In [ ]:
ner_feature = raw_datasets["train"].features["ner_tags"]
ner_feature

Sequence(feature=ClassLabel(names=['O', 'B-PROFANITY', 'I-PROFANITY', 'B-FEEDBACK', 'I-FEEDBACK', 'B-GENERAL', 'I-GENERAL', 'B-VIOLENCE', 'I-VIOLENCE'], id=None), length=-1, id=None)

Check the labels in the dataset

In [ ]:
label_names = ner_feature.feature.names
label_names # GENERAL = slightly negative or positive connotation giving word

['O',
 'B-PROFANITY',
 'I-PROFANITY',
 'B-FEEDBACK',
 'I-FEEDBACK',
 'B-GENERAL',
 'I-GENERAL',
 'B-VIOLENCE',
 'I-VIOLENCE']

Display the token and labels

In [ ]:
words = raw_datasets["train"][0]["tokens"]
labels = raw_datasets["train"][0]["ner_tags"]
line1 = ""
line2 = ""
for word, label in zip(words, labels):
    full_label = label_names[label]
    max_length = max(len(word), len(full_label))
    line1 += word + " " * (max_length - len(word) + 1)
    line2 += full_label + " " * (max_length - len(full_label) + 1)

for x, y in zip(line1.split(), line2.split()):
    print(x, '\t', y)

भालुनी 	 B-PROFANITY
सावित्री 	 O
कुकुरनी 	 B-PROFANITY
मिले 	 O
को 	 O
रहेछ 	 O
आजा 	 O
प्रक्षया 	 O
थाहा 	 O
भयो 	 O
निर्माल 	 O
बहिनी 	 O
लाई 	 O
बलत्कार 	 B-VIOLENCE
गर्न 	 O
लगाउने 	 O
यनि 	 O
भलु 	 B-PROFANITY
हरु 	 O
रहेछ 	 O
पहिला 	 O
जाती 	 O
आन्दोलान 	 O
गरे 	 O
को 	 O
थियो 	 O
त्यो 	 O
सबै 	 O
यनि 	 O
हरु 	 O
को 	 O
नाटक 	 O
रहेछ 	 O
आजा 	 O
बल 	 O
थाहा 	 O
भयो 	 O
। 	 O


## Dataset Statistics

In [ ]:
for dataset_type in ['train', 'validation', 'test']:
    ner_tags_dict = {}
    for each in raw_datasets[dataset_type]:
        for label in each['ner_tags']:
            if label_names[label] != 'O':
                curr_label = label_names[label].split('-')[-1]
                if curr_label in ner_tags_dict:
                    ner_tags_dict[curr_label] += 1
                if curr_label not in ner_tags_dict:
                    ner_tags_dict[curr_label] = 1

    print(f"Dataset statistics for {dataset_type}")
    print(ner_tags_dict)
    print('\n')


Dataset statistics for train
{'PROFANITY': 564, 'VIOLENCE': 527, 'GENERAL': 4383, 'FEEDBACK': 828}


Dataset statistics for validation
{'PROFANITY': 84, 'VIOLENCE': 97, 'GENERAL': 615, 'FEEDBACK': 127}


Dataset statistics for test
{'GENERAL': 569, 'PROFANITY': 65, 'FEEDBACK': 109, 'VIOLENCE': 60}




## Tokenization

In [ ]:
from transformers import AutoTokenizer

# model_checkpoint = "NepBERTa/NepBERTa"
# model_checkpoint = "Rajan/NepaliBERT"
# model_checkpoint = "Rajan/nepbertaTorch"
# model_checkpoint = "Sakonii/distilbert-base-nepali"
model_checkpoint = "Sakonii/deberta-base-nepali"
# model_checkpoint = "xlm-roberta-large"
# model_checkpoint = "bert-base-multilingual-uncased"
# model_checkpoint = "cardiffnlp/twitter-roberta-base-hate"

tokenizer = AutoTokenizer.from_pretrained(model_checkpoint) #, add_prefix_space=True)

tokenizer_config.json:   0%|          | 0.00/413 [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.13M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/239 [00:00<?, ?B/s]

In [ ]:
inputs = tokenizer(raw_datasets["train"][0]["tokens"], is_split_into_words=True)
print(inputs.tokens())

['<s>', '▁भालु', 'नी', '▁सावित्री', '▁कुकुर', 'नी', '▁मिले', '▁', 'को', '▁रहेछ', '▁आज', 'ा', '▁प्र', 'क्ष', 'या', '▁थाहा', '▁भयो', '▁निर्', 'मा', 'ल', '▁बहिनी', '▁', 'लाई', '▁बलत्कार', '▁गर्न', '▁लगाउने', '▁', 'य', 'नि', '▁भल', 'ु', '▁', 'हरु', '▁रहेछ', '▁पहिला', '▁जाती', '▁आ', 'न्दो', 'लान', '▁गरे', '▁', 'को', '▁थियो', '▁त्यो', '▁सबै', '▁', 'य', 'नि', '▁', 'हरु', '▁', 'को', '▁नाटक', '▁रहेछ', '▁आज', 'ा', '▁बल', '▁थाहा', '▁भयो', '▁।', '</s>']


In [ ]:
print(inputs.word_ids())

[None, 0, 0, 1, 2, 2, 3, 4, 4, 5, 6, 6, 7, 7, 7, 8, 9, 10, 10, 10, 11, 12, 12, 13, 14, 15, 16, 16, 16, 17, 17, 18, 18, 19, 20, 21, 22, 22, 22, 23, 24, 24, 25, 26, 27, 28, 28, 28, 29, 29, 30, 30, 31, 32, 33, 33, 34, 35, 36, 37, None]


## Data Preprocessing

In [ ]:
# Align the number of labels and the tokens
def align_labels_with_tokens(labels, word_ids):
    new_labels = []
    current_word = None
    for word_id in word_ids:
        if word_id != current_word:
            # Start of a new word!
            current_word = word_id
            label = -100 if word_id is None else labels[word_id]
            new_labels.append(label)
        elif word_id is None:
            # Special token
            new_labels.append(-100)
        else:
            # Same word as previous token
            label = labels[word_id]
            # If the label is B-XXX we change it to I-XXX
            if label % 2 == 1:
                label += 1
            new_labels.append(label)

    return new_labels

In [ ]:
labels = raw_datasets["train"][0]["ner_tags"]
word_ids = inputs.word_ids()
print(labels)
print(align_labels_with_tokens(labels, word_ids))

[1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 7, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
[-100, 1, 2, 0, 1, 2, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 7, 0, 0, 0, 0, 0, 1, 2, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, -100]


In [ ]:
# Helper function to tokenize and align labels
def tokenize_and_align_labels(examples):
    tokenized_inputs = tokenizer(
        examples["tokens"], truncation=True, is_split_into_words=True
    )
    all_labels = examples["ner_tags"]
    new_labels = []
    for i, labels in enumerate(all_labels):
        word_ids = tokenized_inputs.word_ids(i)
        new_labels.append(align_labels_with_tokens(labels, word_ids))

    tokenized_inputs["labels"] = new_labels
    return tokenized_inputs

In [ ]:
# Tokenize all the examples from the datasets
original_datasets = raw_datasets.map(
    tokenize_and_align_labels,
    batched=True,
)

tokenized_datasets = raw_datasets.map(
    tokenize_and_align_labels,
    batched=True,
    remove_columns=raw_datasets["train"].column_names,
)

Map:   0%|          | 0/2323 [00:00<?, ? examples/s]

Map:   0%|          | 0/330 [00:00<?, ? examples/s]

Map:   0%|          | 0/280 [00:00<?, ? examples/s]

Map:   0%|          | 0/2323 [00:00<?, ? examples/s]

Map:   0%|          | 0/330 [00:00<?, ? examples/s]

Map:   0%|          | 0/280 [00:00<?, ? examples/s]

# Fine Tuning

## Data Collation

Prepare the dataloader for the training session

In [ ]:
from transformers import DataCollatorForTokenClassification

data_collator = DataCollatorForTokenClassification(tokenizer=tokenizer)

batch = data_collator([tokenized_datasets["train"][i] for i in range(2)])

You're using a XLMRobertaTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


In [ ]:
for i in range(2):
    print(tokenized_datasets["train"][i]["labels"])

[-100, 1, 2, 0, 1, 2, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 7, 0, 0, 0, 0, 0, 1, 2, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, -100]
[-100, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 5, 6, 6, 6, 6, 0, 0, 0, 0, 0, 0, 0, 0, -100]


## Setup Evaluation

In [ ]:
import evaluate
import numpy as np

metric = evaluate.load("seqeval")

from sklearn.metrics import classification_report

def compute_metrics(eval_preds):
    logits, labels = eval_preds
    predictions = np.argmax(logits, axis=-1)

    # Remove ignored index (special tokens) and convert to labels
    true_labels = [[label_names[l] for l in label if l != -100] for label in labels]
    true_predictions = [
        [label_names[p] for (p, l) in zip(prediction, label) if l != -100]
        for prediction, label in zip(predictions, labels)
    ]
    all_metrics = metric.compute(predictions=true_predictions, references=true_labels)
    return {
        "precision": all_metrics["overall_precision"],
        "recall": all_metrics["overall_recall"],
        "f1": all_metrics["overall_f1"],
        "accuracy": all_metrics["overall_accuracy"],
    }

In [ ]:
id2label = {i: label for i, label in enumerate(label_names)}
label2id = {v: k for k, v in id2label.items()}

In [ ]:
from transformers import AutoModelForTokenClassification

model = AutoModelForTokenClassification.from_pretrained(
    model_checkpoint,
    id2label=id2label,
    label2id=label2id,
    from_tf=True if model_checkpoint == "NepBERTa/NepBERTa" else False
)


config.json:   0%|          | 0.00/773 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/557M [00:00<?, ?B/s]

Some weights of DebertaForTokenClassification were not initialized from the model checkpoint at Sakonii/deberta-base-nepali and are newly initialized: ['classifier.weight', 'classifier.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
model.config.num_labels

9

## Training

In [ ]:
# Uncomment this when using Colab

# from google.colab import userdata
# from huggingface_hub import login, notebook_login

# # notebook_login()
# login(token=userdata.get('hugging_face'))

In [ ]:
from transformers import TrainingArguments, Trainer
from transformers import EarlyStoppingCallback, IntervalStrategy

model_name = "nepsa-ner"

args = TrainingArguments(
    model_name,
    evaluation_strategy="epoch",
    learning_rate=2e-5,
    num_train_epochs=6,
    weight_decay=0.01,
    push_to_hub=False,
    per_device_train_batch_size=16,
    gradient_accumulation_steps=16
)

trainer = Trainer(
    model=model,
    args=args,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["validation"],
    data_collator=data_collator,
    compute_metrics=compute_metrics,
    tokenizer=tokenizer,
)

trainer.train()

Epoch,Training Loss,Validation Loss,Precision,Recall,F1,Accuracy
1,0.604400,0.450121,0.235977,0.261803,0.248220,0.838560
2,0.401900,0.427667,0.344758,0.366953,0.355509,0.862017
3,0.292400,0.490673,0.338542,0.418455,0.374280,0.848470
4,0.222200,0.496938,0.362162,0.431330,0.393732,0.860763
5,0.166600,0.573321,0.367021,0.444206,0.401942,0.857250
6,0.126200,0.609969,0.364286,0.437768,0.397661,0.856121


TrainOutput(global_step=3486, training_loss=0.2738699100514082, metrics={'train_runtime': 880.5874, 'train_samples_per_second': 15.828, 'train_steps_per_second': 3.959, 'total_flos': 265554476897832.0, 'train_loss': 0.2738699100514082, 'epoch': 6.0})

In [ ]:
trainer.evaluate()

{'eval_loss': 0.6099690794944763,
 'eval_precision': 0.36428571428571427,
 'eval_recall': 0.43776824034334766,
 'eval_f1': 0.39766081871345027,
 'eval_accuracy': 0.8561214249874561,
 'eval_runtime': 1.5216,
 'eval_samples_per_second': 216.874,
 'eval_steps_per_second': 27.602,
 'epoch': 6.0}

## Hyperparameter optimization

Uncomment this to perform hyperparameter optimization.

In [ ]:
# !pip install optuna

In [ ]:
# def model_init():
#     return AutoModelForTokenClassification.from_pretrained(
#     model_checkpoint,
#     id2label=id2label,
#     label2id=label2id,
#     # from_tf=True,
#     )

# trainer = Trainer(
#     model_init=model_init,
#     args=args,
#     train_dataset=tokenized_datasets["train"],
#     eval_dataset=tokenized_datasets["validation"],
#     data_collator=data_collator,
#     tokenizer=tokenizer,
#     compute_metrics=compute_metrics
# )

# def optuna_hp_space(trial):
#     return {
#         "learning_rate": trial.suggest_float("learning_rate", 1e-6, 1e-4, log=True),
#         "per_device_train_batch_size": trial.suggest_categorical("per_device_train_batch_size", [16, 32, 64, 128]),
#     }

# best_trial = trainer.hyperparameter_search(
#     direction="maximize",
#     backend="optuna",
#     hp_space=optuna_hp_space,
#     n_trials=2,
#     # compute_objective=compute_objective,
# )

## Evaluation

In [ ]:
test_predictions = trainer.predict(tokenized_datasets["test"])

from tabulate import tabulate

metrics = ['precision', 'recall', 'f1', 'accuracy']
test_prediction_results = []

for key, val in test_predictions.metrics.items():
    if any(item in key for item in metrics):
        test_prediction_results.append([key, str(round(val,4)*100)+'%'])

print(tabulate(test_prediction_results, headers=['Metric', 'Score']))

Metric          Score
--------------  -------
test_precision  37.92%
test_recall     42.82%
test_f1         40.22%
test_accuracy   86.32%


## Inference a random text from Youtube video

In [ ]:
from transformers import pipeline

latest_model = os.path.join(model_name, 'checkpoint-500')

token_classifier = pipeline(
    "token-classification", model=latest_model, aggregation_strategy="simple"
)


In [ ]:
results = token_classifier("वा जैनु जि तपाइलाइ धन्यबाद छ गगन्या चोर लाई मुख भरी जबाब दिएको मा")
#ओली दलाल मुर्दाबाद
#यो गोविन्दे लाई देश निकाला गर्नु पर्छ")
#यो मुला गोबिन्द ठिक छैन
#यो गोविन्दे लाई देश निकाला गर्नु पर्छ"
 #यो पुण्य गौतम जड्या हो जस्तो कस कस लाई लाग्छ ।")

prediction_results = []
for each_entity in results:
    prediction_results.append([each_entity['word'], each_entity['entity_group']])

print(tabulate(prediction_results, headers=['Aspect Term', 'Predicted Aspect Category']))


Aspect Term    Predicted Aspect Category
-------------  ---------------------------
धन्यबाद          GENERAL


# Inference on example from test dataset

In [ ]:
example_num = 20

decoded_tokens = tokenizer.decode(tokenized_datasets['test'][example_num]['input_ids'], skip_special_tokens=True)
results = token_classifier(decoded_tokens)

prediction_results = []
ground_truth = []


for each_entity in results:
    prediction_results.append([each_entity['word'], each_entity['entity_group']])

for token, tag in zip(decoded_tokens.split(), original_datasets['test'][example_num]['ner_tags']):
    ground_truth.append([token, id2label[tag]])

print(tabulate(prediction_results, headers=['Aspect Term', 'Prediction']))
print('\n')
print(tabulate(ground_truth, headers=['Aspect Term', 'Ground Truth']))

Aspect Term    Prediction
-------------  ------------
जिन्दावाद          GENERAL


Aspect Term    Ground Truth
-------------  --------------
कुलमान           O
सर             O
जिन्दावाद          B-GENERAL
️️️               O
को              O
को              O
हुन्छ            O
कुलमान           O
को              O
पसोट            O
मा              O
।              O
